This is a hacky script to get the timing data from the Psychopy scripts into AFNI-friendly format
for TechScan_900. I can make this a bit less hacky, but the big take-home is that changes to logging in
PsychoPy will make this a lot easier. Right now the start times for the audio trials are not logged.
Also, the keypresses are not logged in the csv file. 

In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
Rootdir = '/Users/handwerkerd/Downloads/psychopy' # very hacky, just for testing
sbjid = '900'
Runnums = [1, 2, 3]
os.chdir(Rootdir)

In [3]:
ProcTypes = np.array(['VisProc', 'FalVisProc', 'AudProc', 'FalAudProc'])
ShowExpected=True # Adds more print lines to show measured vs expected timings
NumTrials=12 # There should be 12 trials per condition
for ProcType in ProcTypes:
    TrialTimes = np.empty((NumTrials, len(Runnums)))
    for runidx in range(3):
        Runnum = Runnums[runidx]
        csvfilenames = glob.glob(f"{sbjid}_WordNonword_fastloc_RUN{Runnum}_????_???_??_????.csv")
        if len(csvfilenames)>1:
            print(f"Error: More than one file: {csvfilenames}")
        else:
            rundata = pd.read_csv(csvfilenames[0])
            # For each trial type load the measured time the first stimulus appeared in a trial (stim1_time)
            #   and when that stimulus was supposed to apper (StartSec)
            TrialTimes[:,runidx] = np.around(rundata.loc[rundata['Procedure']==ProcType,'stim1_time'].values,decimals=1)
            ExpectedStart = np.around(rundata.loc[rundata['Procedure']==ProcType,'StartSec'].values,decimals=1)
            if ShowExpected:
                print(f"Run {Runnum} {ProcType}")
                print(f"Actual Times:   {TrialTimes[:,runidx]}")
                print(f"Expected Times: {ExpectedStart}")
    # Save a separate .1D file for each trial type
    np.savetxt(f"{ProcType}_Times.1D", TrialTimes.T, fmt='%4.1f')



Run 1 VisProc
Actual Times:   [ 13.5  24.   57.  108.  147.  198.  220.5 231.  328.5 381.  442.5 460.5]
Expected Times: [ 13.5  24.   57.  108.  147.  198.  220.5 231.  328.5 381.  442.5 460.5]
Run 2 VisProc
Actual Times:   [ 13.5  27.   52.5 115.5 124.5 219.  268.5 279.  316.5 328.5 406.5 417. ]
Expected Times: [ 13.5  27.   52.5 115.5 124.5 219.  268.5 279.  316.5 328.5 406.5 417. ]
Run 3 VisProc
Actual Times:   [102.  150.  199.5 211.5 289.5 309.  349.5 403.5 420.  474.  492.  502.5]
Expected Times: [102.  150.  199.5 211.5 289.5 309.  349.5 403.5 420.  474.  492.  502.5]
Run 1 FalVisProc
Actual Times:   [ 87.  117.  156.  165.  240.  259.5 276.  303.  400.5 412.5 474.  484.5]
Expected Times: [ 87.  117.  156.  165.  240.  259.5 276.  303.  400.5 412.5 474.  484.5]
Run 2 FalVisProc
Actual Times:   [ 36.   61.5  82.5 103.5 133.5 228.  288.  367.5 388.5 397.5 474.  483. ]
Expected Times: [ 36.   61.5  82.5 103.5 133.5 228.  288.  367.5 388.5 397.5 474.  483. ]
Run 3 FalVisProc
Actual 

In [4]:
# Any of the following 4 buttons is considered a response
ButtonKeys = ['Keypress: r', 'Keypress: b', 'Keypress: g', 'Keypress: y']
KeypressTimesAllRuns = []
for runidx in range(3):
    triggertimes = np.empty(1000) # timestamps the trigger 't' was pressed
    keypresstimes = np.empty(1000) # timestamps a button was pressed
    ExpectedResponseTimes = np.empty(1000) # When the 4th stimulus appeared in a trial (after which a response might be expected)
    Runnum = Runnums[runidx]
    # The 4th stimulus times are only in the csv file and the other times are in the log file
    csvfilenames = glob.glob(f"{sbjid}_WordNonword_fastloc_RUN{Runnum}_????_???_??_????.csv")
    if len(csvfilenames)>1:
        print(f"Error: More than one file: {csvfilenames}")
    else:
        rundata = pd.read_csv(csvfilenames[0])
        logfilenames = glob.glob(f"{sbjid}_WordNonword_fastloc_RUN{Runnum}_????_???_??_????.log")
        logdata = pd.read_csv(logfilenames[0], sep='\t')

    # Find the times when the 4th stimulus appeared for each trial
    Stim4Times = rundata.loc[rundata['Procedure'] != 'Null','stim4_time'].values
    Stim4Times = Stim4Times[~np.isnan(Stim4Times)]

    logvals = logdata.iloc[:,2].values
    trigidx = 0
    keypressidx = 0
    ntidx = 0
    for lidx in range(len(logvals)):
        # For the log lines that initialize each trial, if there's
        # "('Expected_Response', 1)" then a button press is expected
        # Identify those trials and log their trial number "TrialN"
        if 'New trial (' in logvals[lidx]:
            tmp = logdata.iloc[lidx,2]
            tmpidx = np.char.find(tmp, 'Expected_Response')
            tmpbool = tmp[tmpidx+20]
            if tmpbool == '1':
                Rtrialstartidx = np.char.find(tmp,'TrialN')
                Rtrialendidx = np.char.find(tmp, "), ('ProcedureID'")
                tmpTrialNumber = float(tmp[(Rtrialstartidx+9):Rtrialendidx])
                print(tmpTrialNumber)
                ExpectedResponseTimes[ntidx] = Stim4Times[int(tmpTrialNumber-1)]
                ntidx+=1

        if 'Keypress: t' in logvals[lidx]:
            triggertimes[trigidx] = logdata.iloc[lidx,0]
            trigidx+=1
        if any(x in logvals[lidx] for x in ButtonKeys):
            keypresstimes[keypressidx] = logdata.iloc[lidx,0]
            keypressidx+=1

    # Truncate the hacky extra long arrays I stated with
    ExpectedResponseTimes = ExpectedResponseTimes[:ntidx]
    triggertimes = triggertimes[:trigidx]
    # The difference between every 2 trigger pulses shoudl be the TR (with error form keyboard sampling variation)
    VolSpacing = triggertimes[1:]-triggertimes[:-1]

    # keypresstimes are a global clock while the STime4Times are initialized
    # by the first TR. Subtract the first trigger time so they are on the same clock
    keypresstimes = keypresstimes[:keypressidx]-triggertimes[0]

    # Go through all trials where a response is expected and count trials with or without a response
    MissedResponseCount = 0
    CorrectResponseCount = 0
    for eidx in range(len(ExpectedResponseTimes)):
        tmp = np.any((keypresstimes >= (ExpectedResponseTimes[eidx]-0.5)) & (keypresstimes <= (3+ExpectedResponseTimes[eidx])))
        if tmp:
            CorrectResponseCount += 1
        else:
            MissedResponseCount += 1

    # print(f"Keypress Times {keypresstimes}")
    # print(f"Expected Response Times {ExpectedResponseTimes}")

    # Go through all keypresses and log the key presses that are correct or incorrect
    # Note: For now, ALL key presses are given to the GLM in one regressor, but
    #   tracking correct and incorrect responses is useful to gauge attention level
    CorrectKeypressTimes = np.empty(len(keypresstimes))
    IncorrectKeypressTimes = np.empty(len(keypresstimes))
    cidx=0
    iidx=0
    for kidx in range(len(keypresstimes)):
        tmp = np.any(((ExpectedResponseTimes-0.25)<=keypresstimes[kidx]) & (ExpectedResponseTimes>=(keypresstimes[kidx]-3)))
        if tmp:
            CorrectKeypressTimes[cidx] = keypresstimes[kidx]
            cidx += 1
        else:
            IncorrectKeypressTimes[iidx] = keypresstimes[kidx]
            iidx += 1
    CorrectKeypressTimes = CorrectKeypressTimes[:cidx]
    IncorrectKeypressTimes = IncorrectKeypressTimes[:iidx]
    print(f"VolumeSpacing (TR): mean {np.mean(VolSpacing)} std {np.std(VolSpacing)} absmax {np.max(np.abs(VolSpacing-1.5))}")
    print(f"Correct Keypress Times: {CorrectKeypressTimes}")
    print(f"Incorrect Keypress Times: {IncorrectKeypressTimes}")
    print(f"Correct, Missed, Incorrect Keypress Count: {CorrectResponseCount} {MissedResponseCount} {len(IncorrectKeypressTimes)}")
    KeypressTimesAllRuns.append(keypresstimes)

# Save all the keypress times
with open("Keypress_Times.1D", mode='a') as keyfile:
    for runidx in range(len(Runnums)):
        np.savetxt(keyfile, KeypressTimesAllRuns[runidx], fmt='%4.1f', newline="\t")
        np.savetxt(keyfile, [''], fmt='%s')



4.0
9.0
13.0
14.0
17.0
21.0
45.0
46.0
VolumeSpacing (TR): mean 1.5000279411764705 std 0.004562495627339228 absmax 0.019700000000000273
Correct Keypress Times: [ 48.9986 103.131  139.0972 150.9479 179.3802 224.2459 477.643  488.3097]
Incorrect Keypress Times: [ 40.1323  80.2316 243.5966 374.9462 403.2936 426.8792]
Correct, Missed, Incorrect Keypress Count: 8 0 6
7.0
16.0
20.0
38.0
40.0
41.0
43.0
48.0
VolumeSpacing (TR): mean 1.5000264705882351 std 0.004765402817586914 absmax 0.019599999999996953
Correct Keypress Times: [ 86.3979 183.7639 222.9134 401.3778 420.7268 506.1423]
Incorrect Keypress Times: [215.2638 341.3945 371.9939 382.6445]
Correct, Missed, Incorrect Keypress Count: 6 2 4
4.0
8.0
22.0
25.0
28.0
39.0
41.0
45.0
VolumeSpacing (TR): mean 1.500029411764706 std 0.004473827773629402 absmax 0.018999999999977035
Correct Keypress Times: [ 50.416   96.8986 234.7966 271.9626 303.9129 423.8274 441.7955 477.8773]
Incorrect Keypress Times: [ 31.0498  38.8326 133.9977 484.8439]
Correct, Mi